## Cleaning the data in preparation for modeling
Our goal: Figure out what makes an intersection dangerous for pedestrians and cyclists. 

<b>Approach/model 1: </b><br>
Look at the number of collisions at each intersection and quantify how much each feature of the intersection (eg, pavement, crosswalk, etc) contributes to having a high number of collisions.
Implementation: Pandas group by intersection to get number of collisions, then fit regression model on the features in the df that corresponds to characteristics of that intersection. 

Potential issues: no marker for number of people using intersection, which might be an important characteristic to note. Danger can then be defined as #collisions/#people using intersection. Some intersections with high traffic might have lower collisions rates than others.   

<b>Approach/model 2:</b><br>
Look at other features that are highly correlated with collisions. These can be independent of the intersection (weather, ligh levels, etc). While some of these cannot be improved upon by the city, others can be (eg, low light levels mitigated by street lights). There should be some grouping by intersections to see if some intersections have a disproportionate amount of collisions due to these effects. 

### Cleaning
The first part of data cleaning is to accurately measure the number of ped-car and cyclist-car collisions. Approximately 9% of the data doesn't have a ped, car, OR cyclist count, which inhibits our ability to accurately categorize accidents. 

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from pandas_profiling import ProfileReport


# Set style and settings
plt.style.use('ggplot')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 15)

In [25]:
collisions = pd.read_csv('../data/external/Collisions.csv',
                        parse_dates={'Datetime': ['INCDTTM']},
                        infer_datetime_format=True)

collisions = (
    collisions.set_index('Datetime')
    .sort_index()
    .drop(columns=['EXCEPTRSNDESC', 'EXCEPTRSNCODE', 'REPORTNO', 'STATUS'])
)

### Filling in missing data
Approximately 9% of the data is missing the persons/cars involved. We can fill this is by looking at the SDOT descriptions of the accidents

In [26]:
# How many of these involve ZERO people (ie, terrible book-keeping)
no_people = collisions.loc[(collisions['PEDCOUNT'] == 0) & 
           (collisions['PEDCYLCOUNT'] == 0) & 
           (collisions['PERSONCOUNT'] == 0) & 
            (collisions['VEHCOUNT'] == 0)]

people = collisions.loc[(collisions['PEDCOUNT'] != 0) | 
           (collisions['PEDCYLCOUNT'] != 0) | 
           (collisions['PERSONCOUNT'] != 0) | 
            (collisions['VEHCOUNT'] != 0)]

print('Fraction of data with no people involved: ', no_people.shape[0]/collisions.shape[0])

Fraction of data with no people involved:  0.08778965323268431


In [27]:
# Make dictionary with the descriptions and counts for each description
description_series = no_people['SDOT_COLDESC'].value_counts()
descriptions = list(description_series.index)
counts = list(description_series)
d = {descriptions[i]:counts[i] for i in range(len(descriptions))}
d

{'NOT ENOUGH INFORMATION / NOT APPLICABLE': 8060,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE': 4765,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END': 3963,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE SIDESWIPE': 894,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE AT ANGLE': 529,
 'MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT': 306,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE SIDESWIPE': 184,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE AT ANGLE': 168,
 'MOTOR VEHCILE STRUCK PEDESTRIAN': 159,
 'MOTOR VEHICLE STRUCK OBJECT IN ROAD': 148,
 'MOTOR VEHICLE STRUCK PEDALCYCLIST, FRONT END AT ANGLE': 67,
 'PEDALCYCLIST STRUCK MOTOR VEHICLE FRONT END AT ANGLE': 24,
 'MOTOR VEHICLE OVERTURNED IN ROAD': 22,
 'DRIVERLESS VEHICLE RAN OFF ROAD - HIT FIXED OBJECT': 10,
 'PEDALCYCLIST STRUCK MOTOR VEHICLE LEFT SIDE SIDESWIPE': 7,
 'DRIVERLESS VEHICLE STRUCK MOTOR VEHICLE REAR END': 7,
 'PEDALCYCLIST OVERTURNED IN ROAD': 6,
 'PEDALCYCLIST STRUCK PEDESTRIAN': 5,
 'DRIVERLESS

In [28]:
# Replace the values in no_people df and merge with original collisions set
veh_count = no_people['SDOT_COLDESC'].apply(lambda x: x.count('VEHICLE') if type(x) == str else 0)
ped_count = no_people['SDOT_COLDESC'].apply(lambda x: x.count('PEDESTRIAN') if type(x) == str else 0)
cyclist_count = no_people['SDOT_COLDESC'].apply(lambda x: x.count('PEDALCYCLIST') if type(x) == str else 0)

no_people.loc[:,'VEHCOUNT'] = veh_count
no_people.loc[:,'PEDCOUNT'] = ped_count
no_people.loc[:, 'PEDCYLCOUNT'] = cyclist_count

/home/maureen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [29]:
# Merge the people and no_people dataframes
df = pd.concat([no_people, people])

### Dealing with NaNs and binary Y/N

In [30]:
# Assigning 0/1 to binary features
df = (
    pd.get_dummies(df, columns=['SPEEDING', 'INATTENTIONIND', 'HITPARKEDCAR', 'PEDROWNOTGRNT'])
    .drop(columns=['HITPARKEDCAR_N'])
)

In [31]:
# Fixing alcohol influence
print(df.shape)
print(df['UNDERINFL'].value_counts())
print(df['UNDERINFL'].isna().sum())

# Drop NA (~ 10% of data)
df.dropna(subset=['UNDERINFL'], inplace=True)

# Fix labeling
df.loc[:,'UNDERINFL'] = df['UNDERINFL'].apply(lambda x: 1 if (x=='Y') or (x==1) else 0)

(220436, 35)
N    103000
0     81676
Y      5398
1      4230
Name: UNDERINFL, dtype: int64
26132


### Dealing with weather, road conditions, and light
Grouping into fewer categories. Any model using a RF will end up building a sparse tree if our categorical variables have high cardinality. This can bias the RF towards the continuous variables. Can also OHE. 

Weather Conditions:<br>
0: Unknown or NaN <br>
1: Clear and Overcast<br>
2: All others (rain/sleet/snow/fog)<br>

Road Conditions:<br>
0: Unknown or NaN<br>
1: Dry<br>
2: All others (wet/sand/mud)<br>


Light Conditions: <br>
0: Unknown or Other or NaN<br>
1: Daylight<br>
2: Dark & no street lights (or lights off<br>
3: Dark (dawn/dusk/street lights)<br>


### Lastly, group severity codes together 

0: Unknown or NaN<br>
1: property damage<br>
2: injury (minor, serious, fatality)<br>

In [32]:
df_temp = df[['WEATHER', 'ROADCOND', 'LIGHTCOND', 'SEVERITYCODE']]
df_temp.head()
df_temp.isnull().sum()

WEATHER         209
ROADCOND        129
LIGHTCOND       298
SEVERITYCODE      1
dtype: int64

In [33]:
def encode_weather(x):
    if (x == 'Unknown') or (x == 0):
        return 'Unknown'
    elif (x == 'Clear') or (x == 'Overcast'):
        return 'Good'
    else:
        return 'Adverse'   

def encode_road(x):    
    if (x == 'Unknown') or (x == 0):
        return 'Unknown'
    elif x == 'Dry':
        return 'Dry'
    else:
        return 'Adverse'
    
def encode_light(x):
    if (x == 'Unknown') or (x == 'Other') or (x == 0):
        return 'Unknown'
    elif x == 'Daylight':
        return 'Daylight'
    elif (x == 'Dark - No Street Lights') or (x == 'Dark - Street Lights Off'):
        return 'VeryDark'
    else:
        return 'Dark'
    
def encode_severity(x):
    """Everything else should return 0 or 1"""
    injury_list = ['2', '2b', '3']
    if x in injury_list:
        return 'Injury'
    elif x == 0:
        return 'Unknown'
    else:
        return 'PropertyDamage'

In [34]:
# Encode weather, road, and light conditions. Nan is filled with 0 prior to OHE

for series, function in zip(['WEATHER', 'ROADCOND', 'LIGHTCOND', 'SEVERITYCODE'], 
                            [encode_weather, encode_road, encode_light, encode_severity]):
    df[series] = df_temp[series].fillna(0).apply(function)
df.head()

,X,Y,OBJECTID,INCKEY,COLDETKEY,ADDRTYPE,INTKEY,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SDOTCOLNUM,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,SPEEDING_Y,INATTENTIONIND_Y,HITPARKEDCAR_Y,PEDROWNOTGRNT_Y
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-01,NaN,NaN,11627,25040,25040,Block,NaN,BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB ...,Injury,Injury Collision,Rear Ended,2,0,0,2,1,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,Adverse,Dry,Dark,4001002.0,13,From same direction - both going straight - bo...,0,0,0,0,0,0
2004-01-01,-122.318760,47.604359,12525,24635,24635,Block,NaN,E ALDER ST BETWEEN 10TH AVE AND 11TH AVE,PropertyDamage,Property Damage Only Collision,Parked Car,2,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",0,Adverse,Adverse,Unknown,4001026.0,32,One parked--one moving,0,0,0,0,0,0
2004-01-01,-122.320780,47.614076,14172,26463,26463,Intersection,29745.0,BROADWAY AND E PIKE ST,Injury,Injury Collision,Left Turn,4,0,0,2,1,0,0,2004/01/01 00:00:00+00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,Good,Adverse,Dark,4001015.0,28,From opposite direction - one left turn - one ...,0,0,0,0,0,0
2004-01-01,-122.383351,47.529183,6538,19530,19530,Block,NaN,SW ROSE ST BETWEEN 39TH AVE SW AND 41ST AVE SW,PropertyDamage,Property Damage Only Collision,Parked Car,2,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,Good,Adverse,Dark,4001014.0,32,One parked--one moving,0,0,0,0,0,0
2004-01-01,-122.329974,47.708637,9665,22520,22520,Block,NaN,N NORTHGATE WAY BETWEEN CORLISS AVE N AND 1ST ...,PropertyDamage,Property Damage Only Collision,Other,1,0,0,1,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),28.0,MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT,0,Adverse,Adverse,Dark,4001024.0,50,Fixed object,0,0,0,0,0,0


In [35]:
# Finally, do OHE for weather, road, and light conditions. 
# Should merge with get dummies above for succintness
df = (
    pd.get_dummies(df, columns=['WEATHER', 'ROADCOND', 'LIGHTCOND', 'SEVERITYCODE'])
)
df.head()

,X,Y,OBJECTID,INCKEY,COLDETKEY,ADDRTYPE,INTKEY,LOCATION,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,UNDERINFL,SDOTCOLNUM,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,SPEEDING_Y,INATTENTIONIND_Y,HITPARKEDCAR_Y,PEDROWNOTGRNT_Y,WEATHER_Adverse,WEATHER_Good,WEATHER_Unknown,ROADCOND_Adverse,ROADCOND_Dry,ROADCOND_Unknown,LIGHTCOND_Dark,LIGHTCOND_Daylight,LIGHTCOND_Unknown,LIGHTCOND_VeryDark,SEVERITYCODE_Injury,SEVERITYCODE_PropertyDamage,SEVERITYCODE_Unknown
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-01,NaN,NaN,11627,25040,25040,Block,NaN,BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB ...,Injury Collision,Rear Ended,2,0,0,2,1,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,4001002.0,13,From same direction - both going straight - bo...,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0
2004-01-01,-122.318760,47.604359,12525,24635,24635,Block,NaN,E ALDER ST BETWEEN 10TH AVE AND 11TH AVE,Property Damage Only Collision,Parked Car,2,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",0,4001026.0,32,One parked--one moving,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
2004-01-01,-122.320780,47.614076,14172,26463,26463,Intersection,29745.0,BROADWAY AND E PIKE ST,Injury Collision,Left Turn,4,0,0,2,1,0,0,2004/01/01 00:00:00+00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,4001015.0,28,From opposite direction - one left turn - one ...,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0
2004-01-01,-122.383351,47.529183,6538,19530,19530,Block,NaN,SW ROSE ST BETWEEN 39TH AVE SW AND 41ST AVE SW,Property Damage Only Collision,Parked Car,2,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,4001014.0,32,One parked--one moving,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0
2004-01-01,-122.329974,47.708637,9665,22520,22520,Block,NaN,N NORTHGATE WAY BETWEEN CORLISS AVE N AND 1ST ...,Property Damage Only Collision,Other,1,0,0,1,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),28.0,MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT,0,4001024.0,50,Fixed object,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0


### Drop some columns
Conservative estimate in case we want to do more with the data



In [36]:
df = df.drop(columns=['SEGLANEKEY', 'SDOTCOLNUM', 'SEVERITYDESC', 
                      'COLLISIONTYPE', 'SDOT_COLDESC', 'COLDETKEY',
                     'ST_COLDESC'])

In [37]:
df.columns

Index(['X', 'Y', 'OBJECTID', 'INCKEY', 'ADDRTYPE', 'INTKEY', 'LOCATION',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES',
       'SERIOUSINJURIES', 'FATALITIES', 'INCDATE', 'JUNCTIONTYPE',
       'SDOT_COLCODE', 'UNDERINFL', 'ST_COLCODE', 'CROSSWALKKEY', 'SPEEDING_Y',
       'INATTENTIONIND_Y', 'HITPARKEDCAR_Y', 'PEDROWNOTGRNT_Y',
       'WEATHER_Adverse', 'WEATHER_Good', 'WEATHER_Unknown',
       'ROADCOND_Adverse', 'ROADCOND_Dry', 'ROADCOND_Unknown',
       'LIGHTCOND_Dark', 'LIGHTCOND_Daylight', 'LIGHTCOND_Unknown',
       'LIGHTCOND_VeryDark', 'SEVERITYCODE_Injury',
       'SEVERITYCODE_PropertyDamage', 'SEVERITYCODE_Unknown'],
      dtype='object')

(59311, 36)

### Save as csv and pkl

In [38]:
# Pickle the dataframe AND save as csv
df.to_pickle('../data/processed/cleaned_data.pkl')
df.to_csv('../data/processed/cleaned_data.csv')